In [2]:
%%time
!pip install -q --upgrade pip fastai

CPU times: user 150 ms, sys: 19.9 ms, total: 170 ms
Wall time: 12.8 s


In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from fastai.vision import *
from fastai.callbacks import CSVLogger

In [5]:
NUMBER_OF_SAMPLES = 50_000
SIZE = 224 // 8 
MAGNIFICATION = 8
INSETS = 2
THRESHOLD = 0.8

NUMBER_OF_SAMPLES * SIZE * SIZE

39200000

# Creating test data

In [6]:
from PIL import ImageDraw as PILImageDraw
from PIL import Image as PILImage

def create_data():
    return np.random.standard_normal(
        size=(NUMBER_OF_SAMPLES, SIZE, SIZE)) > THRESHOLD

def get_label(data):
    return float(np.sum(data, axis=(0,1)))

def render_image(data):
    img = PILImage.new('RGB', (SIZE*MAGNIFICATION, SIZE*MAGNIFICATION), color = (255,255, 255))
    draw = PILImageDraw.Draw(img)

    for (row_index, row) in enumerate(data):
        for (col_index, value) in enumerate(row):
            x0, y0 = row_index*MAGNIFICATION+INSETS, col_index*MAGNIFICATION+INSETS
            x1, y1 = ((row_index+1)*MAGNIFICATION-INSETS)+1, \
                     ((col_index+1)*MAGNIFICATION-INSETS)+1

            if value: 
                draw.rectangle([(x0, y0), (x1, y1)], fill=(255, 153, 0))  
                
    return Image(pil2tensor(img, np.float32).div(255))

class MyImageItemList(ImageList):
    def __init__(self, path, items, **kwargs):
        super().__init__(items, **kwargs)
        self.path = path
    
    def get(self, i):
        return render_image(self.items[i])

In [7]:
%%time
np.random.seed(42)
dt = create_data()
iil = MyImageItemList(Path('.'), items=dt)
ll = iil.split_by_rand_pct(0.2)
ll.label_from_func(get_label)

CPU times: user 2.2 s, sys: 147 ms, total: 2.35 s
Wall time: 2.35 s


# Create CNN

In [ ]:
runs = []

for (epochs, architecture, bs) in [(3, models.resnet152, 70)]:
    runs += [(epochs, 3e-2, 384, bs, architecture,  str(architecture).split()[1])] 

losses = []

for (epochs, lr, size, bs, architecture,  label) in runs:
    print(f'Starting {label}.')
    data_bunch = ll.databunch(bs=bs)
    data_bunch = data_bunch.normalize(imagenet_stats)
    model = cnn_learner(data_bunch, architecture, ps=0, true_wd = False, bn_wd = False, callback_fns=[CSVLogger])
    model.fit_one_cycle(epochs, slice(lr), wd=0)
    model.recorder.plot_losses()
    plt.yscale('log')
    plt.title(label)

    model.recorder.plot_losses()
    plt.yscale('log')
    plt.ylim(0,10)
    plt.title(label)

    model.recorder.plot_losses()
    plt.ylim(0,3)
    plt.title(label)

    losses += [(label, model.recorder.losses, model.recorder.val_losses)]
    model.save('x'+label)

Starting resnet152.


Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /home/ec2-user/.torch/models/resnet152-b121ed2d.pth
241530880it [00:27, 8729830.02it/s] 


epoch,train_loss,valid_loss,time


In [ ]:
labels, train_losses, val_losses = [], [], []
for (label, train_loss, val_loss) in losses:
    labels += [label]
    train_losses += [train_loss[-1].item()]
    val_losses   += [val_loss[-1].item()]

results = pd.DataFrame()
results['label'] = np.array(labels, dtype=np.str)

results['train_loss'] = np.array(train_losses, dtype=np.float32)
results['val_loss'] = np.array(val_losses, dtype=np.float32)
results.sort_values(by='val_loss', inplace=True)
display(results)  

In [ ]:
data_bunch = ll.databunch(bs=bs)
data_bunch = data_bunch.normalize(imagenet_stats)
#model = cnn_learner(data_bunch, architecture, ps=0, true_wd = False, bn_wd = False, callback_fns=[CSVLogger])
model.fit_one_cycle(5, slice(lr), wd=0)
model.recorder.plot_losses()
plt.yscale('log')
plt.title(label)

model.recorder.plot_losses()
plt.yscale('log')
plt.ylim(0,10)
plt.title(label)

model.recorder.plot_losses()
plt.ylim(0,3)
plt.title(label)

losses += [(label, model.recorder.losses, model.recorder.val_losses)]
model.save('x2'+label)

In [ ]:
labels, train_losses, val_losses = [], [], []
for (label, train_loss, val_loss) in losses:
    labels += [label]
    train_losses += [train_loss[-1].item()]
    val_losses   += [val_loss[-1].item()]

results = pd.DataFrame()
results['label'] = np.array(labels, dtype=np.str)

results['train_loss'] = np.array(train_losses, dtype=np.float32)
results['val_loss'] = np.array(val_losses, dtype=np.float32)
results.sort_values(by='val_loss', inplace=True)
display(results)  

In [ ]:
# from fastai.callbacks import CSVLogger
# data_bunch = ll.databunch(bs=12)
# data_bunch = data_bunch.normalize(imagenet_stats)
# model = cnn_learner(data_bunch, models.resnet18, ps=0, true_wd = False, bn_wd = False, callback_fns=[CSVLogger])
# help(model)
# help(model.csv_logger)
# model.fit_one_cycle(1, slice(3e-2), wd=0)
# !ls

In [ ]:
def plot_results(losses, title):
    colors = plt.cm.rainbow(np.linspace(0,1,len(runs)))
    plt.figure(figsize=(16,8))
    plt.ylim(0, 1000)
    for idx, (label, train_loss, val_loss) in enumerate(losses):
        color = colors[idx]
        plt.plot(train_loss, color=color, linestyle="-.", label=label+' train')
        plt.plot(val_loss,   color=color, linestyle="-", label=label+' val')
    plt.legend()
    plt.title(title)
    plt.ylim(0,5)
plot_results(losses, "All")
